In [1]:
import numpy as np
import matplotlib.pylab as plt
from tempfile import NamedTemporaryFile
from IPython.display import HTML
from matplotlib import animation

x = np.linspace(0,1,51)

# Smooth Rectangle
f = 1. / (1. + np.exp(80.*(np.abs(x-0.5) - 0.15)))

# Triangle
#f = np.zeros((51))
#for i in range(13,26):
#    f[i] = (4.)*(x[i]-0.25)
#for i in range(26,38):
#    f[i] = (-4.)*(x[i]-0.75)


exact_f = np.empty((251,51))
f2 = f
exact_f[0,:] = f2
for i in range(1,251):
    if (i%5 == 0):
        f2 = np.roll(f2,2)
    if ((i+50)%125 == 0):
        f2 = np.roll(f2,1)
    exact_f[i,:] = f2

In [3]:
# Fourth Order Spatial, Leapfrog in Time
cnum = 0.4
gamma = 0.025

qc4 = np.empty((251,51))
qf4 = np.empty((251,51))
qc4[0,:] = f
qf4[0,:] = f

# First time step, upwind
qc4[1,0] = qc4[0,0] - cnum*(qc4[0,1]-qc4[0,49])
for j in range(1,50):
    qc4[1,j] = qc4[0,j] - cnum*(qc4[0,j+1]-qc4[0,j-1])
qc4[1,50] = qc4[1,0]
    
for i in range(2,251):
    qc4[i,0] = qf4[i-2,0] - cnum*2.*(8.*(qc4[i-1,1]-qc4[i-1,49]) - qc4[i-1,2] + qc4[i-1,48])/12.
    qc4[i,1] = qf4[i-2,1] - cnum*2.*(8.*(qc4[i-1,2]-qc4[i-1,0]) - qc4[i-1,2] + qc4[i-1,49])/12.
    for j in range(2,49):
        qc4[i,j] = qf4[i-2,j] - cnum*2.*(8.*(qc4[i-1,j+1]-qc4[i-1,j-1]) - qc4[i-1,j+2] + qc4[i-1,j-2])/12.

    # BCs
    qc4[i,49] = qf4[i-2,49] - cnum*2.*(8.*(qc4[i-1,0]-qc4[i-1,48]) - qc4[i-1,1] + qc4[i-1,47])/12.
    qc4[i,50] = qc4[i,0]
    
    # Asselin Filter to control computational mode with Leapfrog in time
    for j in range(51):
        qf4[i-1,j] = qc4[i-1,j] + gamma*(qc4[i,j]-2*qc4[i-1,j]+qf4[i-2,j])
leap4 = qc4
TRER = np.sqrt(np.sum((f - leap4[250,:])**2./50.))

In [4]:
# Fourth Order Spatial, RK3 in Time
cnum = 0.4

qc4 = np.empty((251,51))
rk3_1 = np.empty((251,51))
rk3_2 = np.empty((251,51))
qc4[0,:] = f
rk3_1[0,:] = 0
rk3_2[0,:] = 0
    
for i in range(1,251):
    # First RK3 Step
    rk3_1[i,0] = qc4[i-1,0] - cnum/3.*(8.*(qc4[i-1,1]-qc4[i-1,49]) - qc4[i-1,2] + qc4[i-1,48])/12.
    rk3_1[i,1] = qc4[i-1,1] - cnum/3.*(8.*(qc4[i-1,2]-qc4[i-1,0]) - qc4[i-1,3] + qc4[i-1,49])/12.
    for j in range(2,49):
        rk3_1[i,j] = qc4[i-1,j] - cnum/3.*(8.*(qc4[i-1,j+1]-qc4[i-1,j-1]) - qc4[i-1,j+2] + qc4[i-1,j-2])/12.

    # BCs
    rk3_1[i,49] = qc4[i-1,49] - cnum/3.*(8.*(qc4[i-1,0]-qc4[i-1,48]) - qc4[i-1,1] + qc4[i-1,47])/12.
    rk3_1[i,50] = rk3_1[i,0]
    
    # Second RK3 Step
    rk3_2[i,0] = qc4[i-1,0] - cnum/2.*(8.*(rk3_1[i,1]-rk3_1[i,49]) - rk3_1[i,2] + rk3_1[i,48])/12.
    rk3_2[i,1] = qc4[i-1,1] - cnum/2.*(8.*(rk3_1[i,2]-rk3_1[i,0]) - rk3_1[i,3] + rk3_1[i,49])/12.
    for j in range(2,49):
        rk3_2[i,j] = qc4[i-1,j] - cnum/2.*(8.*(rk3_1[i,j+1]-rk3_1[i,j-1]) - rk3_1[i,j+2] + rk3_1[i,j-2])/12.

    # BCs
    rk3_2[i,49] = qc4[i-1,49] - cnum/2.*(8.*(rk3_1[i,0]-rk3_1[i,48]) - rk3_1[i,1] + rk3_1[i,47])/12.
    rk3_2[i,50] = rk3_2[i,0]
    
    # Third RK3 Step
    qc4[i,0] = qc4[i-1,0] - cnum*(8.*(rk3_2[i,1]-rk3_2[i,49]) - rk3_2[i,2] + rk3_2[i,48])/12.
    qc4[i,1] = qc4[i-1,1] - cnum*(8.*(rk3_2[i,2]-rk3_2[i,0]) - rk3_2[i,3] + rk3_2[i,49])/12.
    for j in range(2,49):
        qc4[i,j] = qc4[i-1,j] - cnum*(8.*(rk3_2[i,j+1]-rk3_2[i,j-1]) - rk3_2[i,j+2] + rk3_2[i,j-2])/12.

    # BCs
    qc4[i,49] = qc4[i-1,49] - cnum*(8.*(rk3_2[i,0]-rk3_2[i,48]) - rk3_2[i,1] + rk3_2[i,47])/12.
    qc4[i,50] = qc4[i,0]

rk4 = qc4
TRER = np.sqrt(np.sum((f - rk4[250,:])**2./50.))

In [7]:
%matplotlib inline
VIDEO_TAG = """<video controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""

def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with NamedTemporaryFile(suffix='.mp4') as f:
            anim.save(f.name, fps=20,dpi=150,extra_args=['-vcodec', 'libx264'])
            video = open(f.name, "rb").read()
        anim._encoded_video = video.encode("base64")
    
    return VIDEO_TAG.format(anim._encoded_video)

def display_animation(anim):
    plt.close(anim._fig)
    return HTML(anim_to_html(anim))

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()
ax = plt.axes(xlim=(0, 1), ylim=(-0.5, 1.5))
plt.title('4th Order in Spatial Diff')
line, = ax.plot([], [], lw=1, label='Leapfrog')
line3, = ax.plot([], [], lw=1,color='darkcyan', label='RK3')
line2, = ax.plot([], [], lw=1,ls='dashed',color='r')
time_template = 't = %.0f'
time_text = ax.text(0.05, 1.4, '')
ax.axvline(0.5,ls='dotted',color='k')
ax.legend(handles=[line,line3],loc='lower center',fontsize=8,ncol=2)
x2 = x

# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    line2.set_data([], [])
    line3.set_data([], [])
    time_text.set_text('')
    return line, line2, line3, time_text

# animation function.  This is called sequentially
def animate(i):
    #x = np.linspace(0, 2, 1000)
    #y = np.sin(2 * np.pi * (x - 0.01 * i))
    line.set_data(x, leap4[i,:])
    line3.set_data(x, rk4[i,:])
    line2.set_data(x, exact_f[i,:])
    time_text.set_text(time_template % (i))
    return line, line2, line3, time_text

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=251, interval=20, blit=True)

# call our new function to display the animation
display_animation(anim)

# To same animation, comment out previous line and
# uncomment the next two lines.
#fname = 'rk3_time_4th_order_space_box.mp4'
#anim.save(fname, fps=20,dpi=500)